In [8]:
from env_merge import myEnv

# the Experiment class is used for running simulations
from flow.core.experiment import Experiment

# the base network class
from flow.networks import Network

# all other imports are standard
from flow.core.params import VehicleParams
from flow.core.params import NetParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.controllers import IDMController, ContinuousRouter, SimLaneChangeController
from flow.core.params import SumoCarFollowingParams, SumoLaneChangeParams, SumoParams

# create some default parameters parameters
env_params = EnvParams()
initial_config = InitialConfig()
vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             #routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             num_vehicles=0)
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             #routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode="strategic",lcpushy=1.0),
             num_vehicles=0)


sim_params = SumoParams(render=True, restart_instance=True, sim_step=1)
sim_params.color_vehicles = False

fileDir='/mnt/c/Users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/merge.net.xml'

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["gneE6","gneE7"]
)
    
# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {
                "gneE7": ["gneE7","gneE6.243"],
                "gneE6": ["gneE6","gneE6.243"]
               }

In [9]:
from flow.core.params import InFlows

inflow = InFlows()

#highway vehicles
inflow.add(veh_type="human",
           edge="gneE6",
           vehs_per_hour=5000,
            depart_lane="random",
            depart_speed="random",
            color="white")

#merging vehicles
inflow.add(veh_type="rl",
           edge="gneE7",
           vehs_per_hour=1000,
            depart_lane="random",
            depart_speed="random",
            color="blue")
inflow.add(veh_type="rl",
           edge="gneE7",
           vehs_per_hour=500,
            depart_lane="random",
            depart_speed="random",
            color="red")

net_params = NetParams(
    template=fileDir,
    inflows=inflow
)

In [ ]:
flow_params = dict(
    exp_tag='merge',
    env_name=myEnv,
    network=Network,
    simulator='traci',
    sim=sim_params,
    env=env_params,
    net=net_params,
    veh=vehicles,
    initial=initial_config,
)

# number of time steps
flow_params['env'].horizon = 10000
exp = Experiment(flow_params)

# run the sumo simulation
_ = exp.run(1)